In [32]:
# standard Python libraries for manipulating strings dates
import os
import re
from datetime import datetime

# lxml/xpath for parsing the web pages
from lxml import html

# Selenium for clicking buttons & pagination
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# pandas for reading into a tabular format & writing to CSV
import pandas as pd

Set up ChromeDriver. We'll need this since we have to click the "Reply" button in order to get the lister's email.

In [21]:
chromedriver = "/Users/rmdelgad/Documents/ChromeDriver/chromedriver.exe"
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)

Visit the New York apartment rental listings start page. Parse out the listing links.

In [22]:
url = 'https://newyork.craigslist.org/d/rooms-shares/search/roo'
driver.get(url)

In [ ]:
page = driver.page_source
tree = html.fromstring(page)

In [ ]:
listing_links = tree.xpath('//a[@class="result-title hdrlnk"]/@href')

Visit the first listing.

In [ ]:
driver.get(listing_links[0])

Click the Reply button and get the lister's contact email.

In [ ]:
reply_button = driver.find_element_by_xpath('//button[@class="reply_button js-only"]')
reply_button.click()

In [38]:
page = driver.page_source
tree = html.fromstring(page)

In [42]:
tree.xpath('//a[contains(@href,"mailto")]/text()')[0]

'h83kp-6334410731@hous.craigslist.org'

Parse out the title text

In [29]:
tree.xpath('//span[@id="titletextonly"]/text()')[0]

'$850 Extravagant rooms available in a 4bedroom renovated room'

Parse out the neighborhood the listing is in

In [30]:
def clean_neighborhood(neighborhood_text): return re.sub(r'[\(\)\s]', '', neighborhood_text)

In [27]:
clean_neighborhood(tree.xpath('//span[@class="postingtitletext"]/small/text()')[0])

'Ridgewood'

Parse out the posting date

In [35]:
def parse_posting_date(posting_datetime): 
    posting_date = re.search(r'^(\d{4}\-\d{2}\-\d{2}).*', posting_datetime).group(1)
    return datetime.strptime(posting_date,'%Y-%m-%d')

In [36]:
parse_posting_date(tree.xpath('//p[@id="display-date"]/time/@datetime')[0])

datetime.datetime(2017, 10, 5, 0, 0)

Remaining tasks are to write code to:
1. wait for certain page elements to load
2. Witerate through the pages, scraping data down to local text files
3. paginate through all of the listing pages to get all 2500 listings
4. read the local text files, parse out the data we need, and write to a CSV file.